<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/clusterGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    F

In [46]:
# Standard libraries
import numpy as np
from scipy import sparse
import seaborn as sns
import pandas as pd
import time
import json
# Plotting libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm
from IPython.display import Javascript  # Restrict height of output cell.

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
from torch_sparse import spmm
# import pyg_lib
import torch_sparse
from torch_geometric.utils import to_networkx
from sklearn.metrics import f1_score

# PyTorch geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid,Amazon
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_geometric import seed_everything
from torch.profiler import profile, record_function, ProfilerActivity
import os.path as osp
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import GraphSAGE
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM
from torch.profiler import profile, record_function, ProfilerActivity

In [21]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")

def f1_micro(data, mask):
    model.eval()
    out = model(data.x.to(device), data.edge_index.to(device))
    pred = out.argmax(dim=1)
    y_true = data.y[mask].cpu().numpy()
    y_pred = pred[mask.to(device)].cpu().numpy()
    return f1_score(y_true, y_pred, average='micro')


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Initial allocated memory: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

Initial allocated memory: 0.00 MB


In [7]:
num_features, data, num_classes, device, dataset = dataset_load()
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')

Using device: cuda


Processing...
Done!


Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [8]:
class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super().__init__()
    self.conv1 = GCNConv(num_features, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, num_classes)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.conv2(x, edge_index)
    return x

In [9]:
def train(data, mask):
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  loss = criterion(out[mask], data.y[mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(data, mask):
  model.eval()
  out = model(data.x.to(device), data.edge_index.to(device))
  pred = out.argmax(dim=1)
  correct = pred[mask.to(device)] == data.y[mask.to(device)]
  acc = int(correct.sum()) / int(mask.sum())
  return acc

In [10]:
data = data.cpu()
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)


Computing METIS partitioning...
Done!


In [22]:
def train_batch(loader):
    model.train()
    total_loss = 0
    i = 1
    for sub_data in train_loader:
        sub_data = sub_data.to(device)
        batch_mem = sum(t.element_size() * t.nelement() for t in [sub_data.x, sub_data.edge_index, sub_data.y])
        print(f"Batch {i}: {batch_mem/1024**2:.2f} MB cluster data")

        out = model(sub_data.x, sub_data.edge_index)
        print(f"  + Forward pass: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
        loss = criterion(out[sub_data.train_mask], sub_data.y[sub_data.train_mask])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        i += 1
    return total_loss / i  # Optional: average loss


In [29]:

model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
data=data.to(device)
start_time= time.time()
for epoch in range(1, 101):
  loss = train_batch(train_loader)
  if epoch % 10 == 0:
    train_acc = test(data, data.train_mask)
    val_acc = test(data, data.val_mask)
    val_f1 = f1_micro(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val F1 (micro): {val_f1:.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


<IPython.core.display.Javascript object>

Batch 1: 9.64 MB cluster data
  + Forward pass: 80.22 MB
Batch 2: 9.71 MB cluster data
  + Forward pass: 80.22 MB
Batch 3: 9.72 MB cluster data
  + Forward pass: 80.22 MB
Batch 4: 9.70 MB cluster data
  + Forward pass: 80.22 MB
Batch 1: 9.75 MB cluster data
  + Forward pass: 80.22 MB
Batch 2: 9.67 MB cluster data
  + Forward pass: 80.22 MB
Batch 3: 9.70 MB cluster data
  + Forward pass: 80.22 MB
Batch 4: 9.67 MB cluster data
  + Forward pass: 80.22 MB
Batch 1: 9.73 MB cluster data
  + Forward pass: 80.22 MB
Batch 2: 9.67 MB cluster data
  + Forward pass: 80.22 MB
Batch 3: 9.65 MB cluster data
  + Forward pass: 80.22 MB
Batch 4: 9.73 MB cluster data
  + Forward pass: 80.22 MB
Batch 1: 9.71 MB cluster data
  + Forward pass: 80.22 MB
Batch 2: 9.65 MB cluster data
  + Forward pass: 80.22 MB
Batch 3: 9.69 MB cluster data
  + Forward pass: 80.22 MB
Batch 4: 9.73 MB cluster data
  + Forward pass: 80.22 MB
Batch 1: 9.67 MB cluster data
  + Forward pass: 80.22 MB
Batch 2: 9.71 MB cluster data
 

In [30]:
test_acc = test(data, data.test_mask)

test_f1 = f1_micro(data, data.test_mask)
print(f'Test F1 Score (micro): {test_f1:.4f}')
print(f'Test test accuracy (micro):  {test_acc:.4f}')

Test F1 Score (micro): 0.7850
Test test accuracy (micro):  0.7850


In [31]:
print(f"Allocated memory after tensor creation: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved memory after tensor creation: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
print(f"Peak allocated memory: {torch.cuda.max_memory_allocated() / (1024**2):.2f} MB")


Allocated memory after tensor creation: 61.80 MB
Reserved memory after tensor creation: 104.00 MB
Peak allocated memory: 80.22 MB


In [37]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "clusterGCN",
    "accuracy": test_acc,
    "f1_micro":test_f1,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":peak_memory_mb
}

with open("clusterGCN_pubmed_results.json", "w") as f:
    json.dump(metrics, f)

**cora dataset**

In [40]:
clean_gpu_memory()

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

num_features, data, num_classes, device, dataset = dataset_load()
data = data.cpu()
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)

model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
data=data.to(device)
start_time= time.time()
for epoch in range(1, 101):
  loss = train_batch(train_loader)
  if epoch % 10 == 0:
    train_acc = test(data, data.train_mask)
    val_acc = test(data, data.val_mask)
    val_f1 = f1_micro(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val F1 (micro): {val_f1:.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
test_acc = test(data, data.test_mask)

test_f1 = f1_micro(data, data.test_mask)
print(f'Test F1 Score (micro): {test_f1:.4f}')
print(f'Test test accuracy (micro):  {test_acc:.4f}')

Memory after cleanup: 71.17 MB
Using device: cuda


Computing METIS partitioning...
Done!


<IPython.core.display.Javascript object>

Batch 1: 3.75 MB cluster data
  + Forward pass: 90.40 MB
Batch 2: 3.72 MB cluster data
  + Forward pass: 90.62 MB
Batch 3: 3.74 MB cluster data
  + Forward pass: 90.62 MB
Batch 4: 3.72 MB cluster data
  + Forward pass: 90.62 MB
Batch 1: 3.75 MB cluster data
  + Forward pass: 90.62 MB
Batch 2: 3.73 MB cluster data
  + Forward pass: 90.62 MB
Batch 3: 3.73 MB cluster data
  + Forward pass: 90.62 MB
Batch 4: 3.72 MB cluster data
  + Forward pass: 90.62 MB
Batch 1: 3.73 MB cluster data
  + Forward pass: 90.62 MB
Batch 2: 3.73 MB cluster data
  + Forward pass: 90.62 MB
Batch 3: 3.74 MB cluster data
  + Forward pass: 90.62 MB
Batch 4: 3.72 MB cluster data
  + Forward pass: 90.62 MB
Batch 1: 3.71 MB cluster data
  + Forward pass: 90.62 MB
Batch 2: 3.74 MB cluster data
  + Forward pass: 90.62 MB
Batch 3: 3.74 MB cluster data
  + Forward pass: 90.62 MB
Batch 4: 3.74 MB cluster data
  + Forward pass: 90.62 MB
Batch 1: 3.73 MB cluster data
  + Forward pass: 90.62 MB
Batch 2: 3.73 MB cluster data
 

In [41]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "clusterGCN",
    "accuracy": test_acc,
    "f1_micro":test_f1,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":peak_memory_mb
}

with open("clusterGCN_Cora_results.json", "w") as f:
    json.dump(metrics, f)

**citeser dataset**

In [42]:
clean_gpu_memory()

def dataset_load():
  print(f"Using device: {device}")
  dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

num_features, data, num_classes, device, dataset = dataset_load()
data = data.cpu()
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)

model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
data=data.to(device)
start_time= time.time()
for epoch in range(1, 101):
  loss = train_batch(train_loader)
  if epoch % 10 == 0:
    train_acc = test(data, data.train_mask)
    val_acc = test(data, data.val_mask)
    val_f1 = f1_micro(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val F1 (micro): {val_f1:.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
test_acc = test(data, data.test_mask)

test_f1 = f1_micro(data, data.test_mask)
print(f'Test F1 Score (micro): {test_f1:.4f}')
print(f'Test test accuracy (micro):  {test_acc:.4f}')

Memory after cleanup: 71.17 MB
Using device: cuda


Processing...
Done!
Computing METIS partitioning...
Done!


<IPython.core.display.Javascript object>

Batch 1: 11.76 MB cluster data
  + Forward pass: 130.96 MB
Batch 2: 11.80 MB cluster data
  + Forward pass: 131.46 MB
Batch 3: 11.79 MB cluster data
  + Forward pass: 131.46 MB
Batch 4: 11.78 MB cluster data
  + Forward pass: 131.46 MB
Batch 1: 11.76 MB cluster data
  + Forward pass: 131.46 MB
Batch 2: 11.77 MB cluster data
  + Forward pass: 131.46 MB
Batch 3: 11.79 MB cluster data
  + Forward pass: 131.46 MB
Batch 4: 11.82 MB cluster data
  + Forward pass: 131.46 MB
Batch 1: 11.79 MB cluster data
  + Forward pass: 131.46 MB
Batch 2: 11.78 MB cluster data
  + Forward pass: 131.46 MB
Batch 3: 11.80 MB cluster data
  + Forward pass: 131.46 MB
Batch 4: 11.76 MB cluster data
  + Forward pass: 131.46 MB
Batch 1: 11.79 MB cluster data
  + Forward pass: 131.46 MB
Batch 2: 11.79 MB cluster data
  + Forward pass: 131.46 MB
Batch 3: 11.76 MB cluster data
  + Forward pass: 131.46 MB
Batch 4: 11.80 MB cluster data
  + Forward pass: 131.46 MB
Batch 1: 11.81 MB cluster data
  + Forward pass: 131.46 

In [43]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "clusterGCN",
    "accuracy": test_acc,
    "f1_micro":test_f1,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":peak_memory_mb
}

with open("clusterGCN_CiteSeer_results.json", "w") as f:
    json.dump(metrics, f)

**Amazon dataset**

In [47]:
clean_gpu_memory()

def dataset_load():
  print(f"Using device: {device}")
  dataset = Amazon(
        root='data/Amazon',
        name='Computers',
        transform=T.Compose([
        NormalizeFeatures(),          # feature‑wise ℓ₂ normalisation
        RandomNodeSplit(              # ⇦ add a split transform
                split='train_rest',       # 10% val, 10% test by default
                num_val=0.1,
                num_test=0.1,
                num_splits=1,
            )
        ])
    )
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

num_features, data, num_classes, device, dataset = dataset_load()
data = data.cpu()
cluster_data = ClusterData(data, num_parts=128)
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)

model = GCN(hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss().to(device)

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
data=data.to(device)
start_time= time.time()
for epoch in range(1, 101):
  loss = train_batch(train_loader)
  if epoch % 10 == 0:
    train_acc = test(data, data.train_mask)
    val_acc = test(data, data.val_mask)
    val_f1 = f1_micro(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val F1 (micro): {val_f1:.4f}')

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")
test_acc = test(data, data.test_mask)

test_f1 = f1_micro(data, data.test_mask)
print(f'Test F1 Score (micro): {test_f1:.4f}')
print(f'Test test accuracy (micro):  {test_acc:.4f}')

Memory after cleanup: 103.76 MB
Using device: cuda


Processing...
Done!
Computing METIS partitioning...
Done!


<IPython.core.display.Javascript object>

Batch 1: 11.10 MB cluster data
  + Forward pass: 173.99 MB
Batch 2: 11.08 MB cluster data
  + Forward pass: 173.99 MB
Batch 3: 11.03 MB cluster data
  + Forward pass: 173.99 MB
Batch 4: 11.13 MB cluster data
  + Forward pass: 175.52 MB
Batch 1: 11.33 MB cluster data
  + Forward pass: 176.58 MB
Batch 2: 11.05 MB cluster data
  + Forward pass: 176.58 MB
Batch 3: 10.83 MB cluster data
  + Forward pass: 176.58 MB
Batch 4: 11.23 MB cluster data
  + Forward pass: 176.58 MB
Batch 1: 11.18 MB cluster data
  + Forward pass: 176.58 MB
Batch 2: 11.34 MB cluster data
  + Forward pass: 176.58 MB
Batch 3: 10.83 MB cluster data
  + Forward pass: 176.58 MB
Batch 4: 10.94 MB cluster data
  + Forward pass: 176.58 MB
Batch 1: 11.06 MB cluster data
  + Forward pass: 176.58 MB
Batch 2: 11.09 MB cluster data
  + Forward pass: 176.58 MB
Batch 3: 11.23 MB cluster data
  + Forward pass: 176.58 MB
Batch 4: 11.00 MB cluster data
  + Forward pass: 176.58 MB
Batch 1: 10.91 MB cluster data
  + Forward pass: 176.58 

In [48]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "clusterGCN",
    "accuracy": test_acc,
    "f1_micro":test_f1,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":peak_memory_mb
}

with open("clusterGCN_Amazon_results.json", "w") as f:
    json.dump(metrics, f)